# 07_Digital_Twin_Views.ipynb

This notebook defines views that represent the **Digital Twin** of each aircraft. A Digital Twin is a virtual representation of an aircraft's state, derived from real-time and historical sensor data. These views support monitoring, simulation, and risk assessment use cases by providing structured, queryable representations of the latest known state of the aircraft and its components.

**Views defined in this notebook include:**
- `digital_twin_aircraft_view`: one record per aircraft representing the latest alert data.
- `digital_twin_engine_view`: a component-level view focused on engine health based on sensor values.

These views are built from the `anomaly_alerts` table which holds the latest high-risk predictions with metadata such as timestamps and batch IDs.

## 1. Aircraft-Level Digital Twin View

This view selects the **most recent alert record** for each aircraft from the `anomaly_alerts` Delta table. It uses `ROW_NUMBER()` to identify the latest record per aircraft based on the `timestamp` column.

This is the base view for the Digital Twin representation.

In [0]:
spark.sql("""
CREATE OR REPLACE VIEW arao.aerodemo.digital_twin_aircraft_view AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY aircraft_id ORDER BY timestamp DESC) AS rn
    FROM arao.aerodemo.anomaly_alerts
) latest
WHERE rn = 1
""")

## 2. Engine Health View (Component-Level)

This view derives the **engine health status** from the Digital Twin aircraft-level view.

It applies business rules to sensor readings (`engine_temp`, `vibration`, `engine_rpm`) and classifies the engine condition into categories:
- `HIGH_RISK`: temp > 700°C or vibration > 5.0
- `MODERATE_RISK`: temp between 600°C and 700°C
- `NORMAL`: anything below that

In [0]:
spark.sql("""
CREATE OR REPLACE VIEW arao.aerodemo.digital_twin_engine_view AS
SELECT
  aircraft_id,
  engine_temp,
  vibration,
  engine_rpm,
  CASE
    WHEN engine_temp > 700 OR vibration > 5 THEN 'HIGH_RISK'
    WHEN engine_temp > 600 THEN 'MODERATE_RISK'
    ELSE 'NORMAL'
  END AS engine_health_status,
  timestamp
FROM arao.aerodemo.digital_twin_aircraft_view
""")

## 3. Next Steps

These views can now be used to:
- Build real-time dashboards for aircraft monitoring
- Run analytical queries to detect patterns and risk exposure
- Trigger alerts or workflows based on health status changes

You can extend this with views for:
- Electrical systems (`battery_voltage`)
- Environmental conditions (`altitude`, `airspeed`)
- Maintenance prediction (`days_since_maint`, `prev_anomaly`)

Continue building your Digital Twin layers based on business needs.